In [2]:
!pip install SpeechRecognition


In [12]:
# Step 1: Install necessary libraries
!pip install SpeechRecognition pydub
!apt-get install ffmpeg -y

import zipfile
import os
import random
import speech_recognition as sr
from pydub import AudioSegment
from google.colab import files

# Step 2: Upload ZIP file
print("Upload your input.zip containing folders and audio files:")
uploaded = files.upload()

zip_path = list(uploaded.keys())[0]
extract_folder = "extracted_audio"

# Step 3: Extract ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f"Extracted files to '{extract_folder}'")

# Step 4: Function to get all audio files recursively
def get_all_audio_files(base_folder, extensions=('.wav', '.mp3')):
    audio_files = []
    for root, dirs, files in os.walk(base_folder):
        for file in files:
            if file.lower().endswith(extensions):
                audio_files.append(os.path.join(root, file))
    return audio_files

# Step 5: Convert any mp3 files to wav for compatibility
audio_files = get_all_audio_files(extract_folder)
for file_path in audio_files:
    if file_path.lower().endswith(".mp3"):
        wav_path = file_path.rsplit('.', 1)[0] + ".wav"
        print(f"Converting {file_path} to {wav_path}")
        sound = AudioSegment.from_mp3(file_path)
        sound.export(wav_path, format="wav")
        os.remove(file_path)  # Remove original mp3

# Step 6: Refresh audio files list to only wav files now
audio_files = get_all_audio_files(extract_folder, extensions=('.wav',))
print(f"Found {len(audio_files)} WAV audio files after conversion.")

if len(audio_files) < 5:
    print("Warning: Less than 5 audio files found, will process all available.")

# Step 7: Select 5 random audio files (or all if less than 5)
num_files = min(5, len(audio_files))
selected_files = random.sample(audio_files, num_files)
print(f"Selected {num_files} audio files:")

for f in selected_files:
    print(f)

# Step 8: Recognizer setup
recognizer = sr.Recognizer()

valid_commands = {
    "shoot", "jump", "reload", "stone", "rock", "paper", "scissors", "scissor"
}

output_lines = []

# Step 9: Process each selected audio file
for idx, audio_path in enumerate(selected_files, 1):
    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)

    try:
        spoken_text = recognizer.recognize_google(audio).lower()
        print(f"[{idx}] Recognized text: {spoken_text}")
    except Exception as e:
        print(f"[{idx}] Could not recognize audio: {str(e)}")
        spoken_text = None

    output_lines.append(f"Audio file [{idx}]: {os.path.basename(audio_path)}")
    if spoken_text in valid_commands:
        # Handle stone-paper-scissors game commands
        if spoken_text in {"stone", "rock", "paper", "scissors", "scissor"}:
            if spoken_text == "stone":
                player_move = "rock"
            elif spoken_text == "scissor":
                player_move = "scissors"
            else:
                player_move = spoken_text

            moves = ["rock", "paper", "scissors"]
            computer_move = random.choice(moves)

            def decide_winner(player, comp):
                if player == comp:
                    return "draw"
                elif (player == "rock" and comp == "scissors") or \
                     (player == "scissors" and comp == "paper") or \
                     (player == "paper" and comp == "rock"):
                    return "player"
                else:
                    return "computer"

            winner = decide_winner(player_move, computer_move)
            output_lines.append(f"Your move: {player_move}")
            output_lines.append(f"Computer move: {computer_move}")
            output_lines.append(f"Winner: {winner}")

        else:
            # Other valid commands (shoot, jump, reload)
            output_lines.append(f"Command recognized: {spoken_text}")

    else:
        output_lines.append("Command not recognized or not a valid game command. Ignored.")
    output_lines.append("")  # blank line for readability

# Step 10: Write output to output.txt
with open("output.txt", "w") as f:
    for line in output_lines:
        f.write(line + "\n")

print("Output saved to output.txt")

# Step 11 (optional): Download the output file
files.download("output.txt")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Upload your input.zip containing folders and audio files:


Saving game_commands (2).zip to game_commands (2).zip
Extracted files to 'extracted_audio'
Found 18 WAV audio files after conversion.
Selected 5 audio files:
extracted_audio/game_commands/stone.wav
extracted_audio/game_commands/game_commands/reload.wav
extracted_audio/game_commands/game_commands/write.wav
extracted_audio/game_commands/game_commands/stone.wav
extracted_audio/game_commands/reload.wav
[1] Recognized text: stone
[2] Recognized text: reload
[3] Could not recognize audio: 
[4] Recognized text: stone
[5] Recognized text: reload
Output saved to output.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>